<h1> Proyecto 3 </h1>

In [1]:
#import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,DoubleType,IntegerType
# UDF
from pyspark.sql.functions import udf
#pandas udf
from pyspark.sql.functions import pandas_udf, PandasUDFType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1589880021320_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#create spar session object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Paso 1-> Convertir los csv en dataframes</h3>

In [3]:
# Cargando los csv 
#cpcc = Casos positivos covid Colombia
cpcc=spark.read.csv('s3://sebasawsbucket/Proyecto 3/datasets/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)
#dcc = Datos covid Colombia 
dcc=spark.read.csv('s3://sebasawsbucket/Proyecto 3/datasets/covid19_colombia.csv',inferSchema=True,header=True)
#cccg = Casos covid confirmados globalmente
cccg=spark.read.csv('s3://sebasawsbucket/Proyecto 3/datasets/time_series_covid19_confirmed_global.csv',inferSchema=True,header=True)
#ccmg = Casos covid muertos globalmente
ccmg=spark.read.csv('s3://sebasawsbucket/Proyecto 3/datasets/time_series_covid19_deaths_global.csv',inferSchema=True,header=True)
#ccrg = Casos covid recuperados globalmente
ccrg=spark.read.csv('s3://sebasawsbucket/Proyecto 3/datasets/time_series_covid19_recovered_global.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Paso 2-> Visualizar los nombres de las columnas y encontrar patrones</h3>

In [4]:
#Columnas cpcc = Casos positivos covid Colombia
cpcc.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [5]:
#Columnas dcc = Datos covid Colombia 
dcc.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [6]:
#Columnas cccg = Casos covid confirmados globalmente
cccg.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20', '4/19/2

In [7]:
#Columnas ccmg = Casos covid muertos globalmente
ccmg.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20', '4/19/2

In [8]:
#Columnas ccrg = Casos covid recuperados globalmente
ccrg.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20', '4/19/2

<h3>Paso 3-> Visualizar la Cantidad de Registros en los dataframes filas*columnas</h3>

In [9]:
#cpcc = Casos positivos covid Colombia
print((cpcc.count(),len(cpcc.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(16295, 16)

In [10]:
#dcc = Datos covid Colombia 
print((dcc.count(),len(dcc.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(16296, 16)

In [11]:
#cccg = Casos covid confirmados globalmente
print((cccg.count(),len(cccg.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(266, 122)

In [12]:
#ccmg = Casos covid muertos globalmente
print((ccmg.count(),len(ccmg.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(266, 122)

In [13]:
#ccrg = Casos covid recuperados globalmente
print((ccrg.count(),len(ccrg.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(253, 122)

<h3>Paso 4-> Visualizar la estructura de los datos</h3>

In [14]:
#cpcc = Casos positivos covid Colombia
cpcc.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [15]:
#dcc = Datos covid Colombia 
dcc.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: string (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Codigo DIVIPOLA: string (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: string (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: string (nullable = true)

In [16]:
#cccg = Casos covid confirmados globalmente
cccg.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 2/1/20: integer (nullable = true)
 |-- 2/2/20: integer (nullable = true)
 |-- 2/3/20: integer (nullable = true)
 |-- 2/4/20: integer (nullable = true)
 |-- 2/5/20: integer (nullable = true)
 |-- 2/6/20: integer (nullable = true)
 |-- 2/7/20: integer (nullable = true)
 |-- 2/8/20: integer (nullable = true)
 |-- 2/9/20: integer (nullable = true)
 |-- 2/10/20: integer (nullable = true)
 |-- 2/11/20: integer (nullable = true)
 

In [17]:
#ccmg = Casos covid muertos globalmente
ccmg.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 2/1/20: integer (nullable = true)
 |-- 2/2/20: integer (nullable = true)
 |-- 2/3/20: integer (nullable = true)
 |-- 2/4/20: integer (nullable = true)
 |-- 2/5/20: integer (nullable = true)
 |-- 2/6/20: integer (nullable = true)
 |-- 2/7/20: integer (nullable = true)
 |-- 2/8/20: integer (nullable = true)
 |-- 2/9/20: integer (nullable = true)
 |-- 2/10/20: integer (nullable = true)
 |-- 2/11/20: integer (nullable = true)
 

In [18]:
#ccrg = Casos covid recuperados globalmente
ccrg.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 2/1/20: integer (nullable = true)
 |-- 2/2/20: integer (nullable = true)
 |-- 2/3/20: integer (nullable = true)
 |-- 2/4/20: integer (nullable = true)
 |-- 2/5/20: integer (nullable = true)
 |-- 2/6/20: integer (nullable = true)
 |-- 2/7/20: integer (nullable = true)
 |-- 2/8/20: integer (nullable = true)
 |-- 2/9/20: integer (nullable = true)
 |-- 2/10/20: integer (nullable = true)
 |-- 2/11/20: integer (nullable = true)
 

<h3>Paso 5-> Identificando comportamiento en Colombia, usando la muestra de medellin</h3>

In [19]:
cpcc.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16295

In [20]:
cpcc.filter(cpcc['Ciudad de ubicación']=='Medellín').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

356

In [21]:
cpcc.filter(cpcc['atención']=='Recuperado').filter(cpcc['Ciudad de ubicación']=='Medellín').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [22]:
cpcc.filter(cpcc['atención']=='Casa').filter(cpcc['Ciudad de ubicación']=='Medellín').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

141

In [23]:
cpcc.filter(cpcc['atención']=='Hospital').filter(cpcc['Ciudad de ubicación']=='Medellín').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8

In [24]:
cpcc.filter(cpcc['atención']=='Fallecido').filter(cpcc['Ciudad de ubicación']=='Medellín').select('Edad','Sexo','Ciudad de ubicación','Fecha de muerte').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-------------------+--------------------+
|Edad|Sexo|Ciudad de ubicación|     Fecha de muerte|
+----+----+-------------------+--------------------+
|  91|   F|           Medellín|2020-04-03T00:00:...|
|  67|   M|           Medellín|2020-04-18T00:00:...|
|  74|   M|           Medellín|2020-04-27T00:00:...|
+----+----+-------------------+--------------------+

In [25]:
cpcc.filter(cpcc['atención']=='Hospital UCI').filter(cpcc['Ciudad de ubicación']=='Medellín').select('atención','Edad','Sexo','Ciudad de ubicación').show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+----+-------------------+
|    atención|Edad|Sexo|Ciudad de ubicación|
+------------+----+----+-------------------+
|Hospital UCI|  56|   M|           Medellín|
|Hospital UCI|  37|   M|           Medellín|
|Hospital UCI|  71|   F|           Medellín|
|Hospital UCI|  80|   M|           Medellín|
+------------+----+----+-------------------+

<h3>Paso 6-> Vamos a revisar la proporción de casos de covid en el mundo con respecto a Colombia </h3>

In [26]:
cccg.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

266

In [27]:
cccg.filter(cccg['Country/Region']=='Colombia').select('Country/Region','5/18/20',).show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|Country/Region|5/18/20|
+--------------+-------+
|      Colombia|  16295|
+--------------+-------+

In [28]:
cccg.filter(cccg['5/18/20']>='16295').select('Country/Region','5/18/20',).show(39)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|      Country/Region|5/18/20|
+--------------------+-------+
|          Bangladesh|  23870|
|             Belarus|  30572|
|             Belgium|  55559|
|              Brazil| 255368|
|              Canada|  24286|
|              Canada|  43636|
|               Chile|  46059|
|               China|  68135|
|            Colombia|  16295|
|             Ecuador|  33582|
|              France| 177554|
|             Germany| 176551|
|               India| 100328|
|           Indonesia|  18010|
|                Iran| 122492|
|             Ireland|  24200|
|              Israel|  16643|
|               Italy| 225886|
|               Japan|  16305|
|              Mexico|  51633|
|         Netherlands|  44141|
|            Pakistan|  42125|
|                Peru|  94933|
|              Poland|  18885|
|            Portugal|  29209|
|               Qatar|  33969|
|             Romania|  17036|
|              Russia| 290678|
|        Saudi Arabia|  57345|
|       

Conclusión-> Hay 266 paises con covid 19, para la ultima fecha del informe, osea el 5/18/20 Colombia tenía en total 16.295 y en el mundo hay 39 países con más casos que colombia, siendo el caso más preocupando Estados Unidos(US) con 1'508.308

In [29]:
#cpcc = Casos positivos covid Colombia
#dcc = Datos covid Colombia 
#cccg = Casos covid confirmados globalmente
#ccmg = Casos covid muertos globalmente
#ccrg = Casos covid recuperados globalmente

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…